In [ ]:
import geopandas as gpd, pandas as pd

## Open census shp

In [ ]:
# read census 2011 shp
read_census_geometries_2011 = gpd.read_file('Census_data/2011/R08_11_WGS84/R08_11_WGS84.shp')

In [ ]:
read_census_geometries_2011

In [ ]:
# read census 2021 shp
read_census_geometries_2021 = gpd.read_file('Census_data/2021_provisional/R08_21_P/SHP/R08_21_WGS84_P.shp')

In [ ]:
read_census_geometries_2021

##  Census data

In [ ]:
read_census_info_2011 = pd.read_csv('Census_data/2011/dati-cpa_2011/Sezioni di Censimento/R08_indicatori_2011_sezioni.csv', sep=";" ,encoding='ISO-8859-1')

In [ ]:
read_census_info_2011

In [ ]:
read_census_info_2021 = pd.read_csv('Census_data/2021_provisional/Dati_regionali_2021/R08_indicatori_2021_sezioni.csv', sep=";" ,encoding='ISO-8859-1')

In [ ]:
read_census_info_2021

## Join census data

In [ ]:
# Main based on the 'SEZ2011' column
df_merged_2021 = read_census_geometries_2011.merge(read_census_info_2021, left_on='SEZ2011', right_on='SEZ2011', how='inner')

In [ ]:
df_merged_2021

## Read buildings data

In [ ]:
gdf_buildings_DB_emilia = gpd.read_file('Buildings_data/volumetria_edifici_uniti.shp')

In [ ]:
gdf_buildings_IGM = gpd.read_file('Buildings_data/IGM_Data/buildings_emili_romagna.shp')

## Clear buildings on a province for more speed

In [ ]:
# loading of provincial borders
gdf_provincial_boundaries = gpd.read_file('Limiti01012023/ProvCM01012023/ProvCM01012023_WGS84.shp')

In [ ]:
# select a single province
gdf_single_province = gdf_provincial_boundaries[gdf_provincial_boundaries['DEN_PROV'] == 'Piacenza']

In [ ]:
# standardize the reference systems
gdf_single_province = gdf_single_province.to_crs(gdf_buildings_DB_emilia.crs)
gdf_buildings_IGM = gdf_buildings_IGM.to_crs(gdf_buildings_DB_emilia.crs)

In [ ]:
# select only one province to speed up the calculation
gdf_sjoin_DB_emilia = gpd.sjoin(gdf_buildings_DB_emilia, gdf_single_province, how="inner", op="intersects")
gdf_sjoin_IGM = gpd.sjoin(gdf_buildings_IGM, gdf_single_province, how="inner", op="intersects")

In [ ]:
# reset gdf index
gdf_sjoin_DB_emilia = gdf_sjoin_DB_emilia.reset_index(drop=True)
gdf_sjoin_IGM = gdf_sjoin_DB_emilia.reset_index(drop=True)

In [ ]:
# Create a new GeoDataFrame with points at the centroids of the polygons
gdf_point = gdf_sjoin_IGM.copy()
gdf_point['geometry'] = gdf_point['geometry'].centroid

In [ ]:
gdf_point.to_file('points.shp')

In [ ]:
gdf_sjoin_DB_emilia = gdf_sjoin_DB_emilia.rename(columns={'index': 'left_index'})
gdf_point = gdf_point.rename(columns={'index': 'right_index'})

In [ ]:
# Perform a spatial join using the "nearest" operation
gdf_all_info = gpd.sjoin(gdf_sjoin_DB_emilia, gdf_point, how="inner", op="nearest")